In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import napari
from skimage.io import imread, imsave
import pickle

In [ ]:
# Append the path of the scripts directory to sys.path
sys.path.append('./src/statistics_collection/')

In [ ]:
from misc import *
from StatsCollector import StatsCollector
from LabelPreprocessing import process_labels, get_labels_touching_edges
from GenMeshes import convert_labels_to_meshes

In [ ]:
os.getcwd()

In [ ]:
from src.tests.CubeLatticeTest import *

labeled_img = generate_cube_lattice_image(
    nb_x_voxels=10,
    nb_y_voxels=10,
    nb_z_voxels=10,
    cube_side_length=3,
    nb_cubes_x=3,
    nb_cubes_y=3,
    nb_cubes_z=3,
    interstitial_space=-1,
)

In [ ]:
viewer = napari.Viewer()
viewer.add_labels(labeled_img)

In [ ]:
smoothing_iters = 10
erosion_iters = 6
dilation_iters = 8

PATH_TO_OUTPUT = 'src/tests/cube_output'
output_dir = create_output_directory(
    output_folder=PATH_TO_OUTPUT,
    smoothing_iterations=smoothing_iters, 
    erosion_iterations=erosion_iters, 
    dilation_iterations=8
)

preprocessed_labeled_img = process_labels(
    labeled_img=labeled_img, 
    erosion_iterations=erosion_iters,
    dilation_iterations=dilation_iters,
    output_directory=output_dir,
    overwrite=False
)

meshes = convert_labels_to_meshes(
    preprocessed_labeled_img,
    [1, 1, 1],
    10,
    output_dir,
    False,
    10,
    'stl'
)

In [ ]:
stats_collector = StatsCollector(
    meshes=meshes,
    labels=preprocessed_labeled_img,
    features=['area', 'volume', 'elongation_and_axes', 'neighbors', 'contact_area'],
    output_directory=output_dir,
    path_to_img=os.path.join(output_dir, 'processed_labels.tif'),
    tissue='lung_bronchiole',
    num_workers=2
)

In [ ]:
stats_collector.collect_statistics()

## Test filtering

In [ ]:
img = imread('output_intestine_sample2_b_curated_segmentation_relabel_seq_s_10_e_6_d_8/processed_labels.tif')
idxs = get_labels_touching_edges(labeled_img=img)

In [ ]:
v = napari.Viewer()
v.add_labels(img)
for id in idxs:
    single_cell = img.copy()
    single_cell[single_cell != id] = 0
    v.add_labels(single_cell, name=f'cell_{id}')

## Test 2D stats collection

In [ ]:
v = napari.Viewer()

In [ ]:
from src.tests.CubeLatticeTest import generate_cube_lattice_image

test_cubes = generate_cube_lattice_image(20, 20, 20, 5, 2, 2, 2, 0)
v.add_labels(test_cubes)
test_cubes.shape

In [ ]:
from StatsUtils import compute_2D_statistics

feat_dicts = compute_2D_statistics(labeled_img=test_cubes, slicing_dim=0, exclude_labels=[], pixel_size=[1, 1])

In [ ]:
import pickle

save_name = f'try_save.pickle'
with open(os.path.join('./images/', save_name), 'wb') as file:
    pickle.dump(feat_dicts, file)

In [ ]:
img = imread('output_lung_new_sample_b_curated_segmentation_central_crop_relabel_seq_s_10_e_6_d_8/processed_labels.tif')
neighbors, areas = compute_2D_statistics(labeled_img=img, slicing_dim=0, exclude_labels=[], pixel_size=[0.1625, 0.1625])

In [ ]:
from StatsCollector import StatsCollector

neigh = StatsCollector._unpack_feature_dict(neighbors)
ar = StatsCollector._unpack_feature_dict(areas)

## Test StatsCollector on cubes

In [ ]:
from src.tests.CubeLatticeTest import generate_cube_lattice_image

test_cubes = generate_cube_lattice_image(20, 20, 20, 5, 3, 3, 3, 0)
imsave('./src/tests/cube_lattice1.tif', test_cubes)

In [ ]:
from run import main

main('./config_test.json')

In [ ]:
cube_df = pd.read_csv('output_cube_lattice1_s_10_e_6_d_8/cell_stats/stats_dataset_embryo.csv', index_col=0)
cube_df

### Test on Embrios data

In [ ]:
# labeled_img = imread('/nas/groups/iber/Projects/Early_Embryo/maoxia/movie_4/curation/cell_boundary_time_1.tif')

# viewer = napari.Viewer()
# viewer.add_labels(labeled_img)

In [ ]:
from run import main

main('./src/statistics_collection/config_test.json')

In [ ]:
embryo_df = pd.read_csv('outputs/output_cell_boundary_time_1_s_10_e_6_d_8/cell_stats/stats_dataset_embryo.csv', index_col=0)
embryo_df

In [ ]:
def chiama_mamma(name):
    print(f'Ciao mamma {name}.')

def auguri_mamma(age):
    print(f'Auguri mamma per i tuoi {age} anni.')

def saluta(methods, *args, **kwargs):
    if 'chiama' in methods:
        chiama_mamma(kwargs.pop('name'))
    if 'auguri' in methods:
        auguri_mamma(kwargs.pop('age'))

In [ ]:
saluta(['chiama', 'auguri'], name='marina', age=57)

## Test contact area computation

In [ ]:
import napari

viewer = napari.Viewer()
img = imread('output_lung_new_sample_b_curated_segmentation_central_crop_relabel_seq_s_10_e_6_d_8/processed_labels.tif')
viewer.add_labels(img, name='Original image')

In [ ]:
from StatsUtils import _compute_contact_area
import trimesh as tm

# Upload required data
path_to_meshes = 'output_intestine_sample2_b_curated_segmentation_relabel_seq_s_10_e_6_d_8/cell_meshes'
mesh_files = sorted(os.listdir(path_to_meshes), key=lambda x: int(x.split('_')[1].split('.')[0]))
mesh_files

meshes_dict = {}
for i, mesh_file in enumerate(mesh_files):
    meshes_dict[i+1] = tm.load_mesh(os.path.join(path_to_meshes, mesh_file))

with open(os.path.join('output_intestine_sample2_b_curated_segmentation_relabel_seq_s_10_e_6_d_8/cell_stats/cached_stats/cell_neighbors.pickle'), 'rb') as file:
    neighbors_dict = pickle.load(file)

In [ ]:
cell_id = 5
neighbors_list = neighbors_dict[cell_id]
neighbors_list

In [ ]:
results = _compute_contact_area(meshes_dict, cell_id, neighbors_list, 0.65, show_napari=True)

In [ ]:
results[0], np.sum(results[1]), meshes_dict[cell_id].area

In [ ]:
viewer = napari.Viewer()
img = imread('output_cell_boundary_time_1_s_10_e_6_d_8/processed_labels.tif')
viewer.add_labels(img, name='Original image')
for id, mesh in meshes_dict.items():
    viewer.add_surface((mesh.vertices, mesh.faces), name=f'Mesh_{id}', blending='additive', opacity=0.6)